# Training an XOR network using PySyft

In this notebook we'll be training an XOR network using PySyft. 
What's special about this is that the training  is DP- Protected


In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
import numpy as np
sy.logger.remove()

### Logging into the domain Nodes

In [2]:
fb = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)

Connecting to localhost... done! 	 Logging into adp... done!


In [3]:
fb.datasets

Idx,Name,Description,Assets,Id
[0],Canada Trade Data - First few rows,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> int64",0731cbfb-818e-4980-8236-6238676bbad7
[1],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",fb3d41c9-1c39-4544-9501-280b6997ff8d
[2],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",de26285c-506b-488e-bfa4-0cd9d62c64bc
[3],Our training data for XOR networks!,Collected on Jan 27 2022,"[""training_data""] -> int64[""training_targets""] -> int64",4c920ecb-9215-4766-82a5-3116df87937a


In [4]:
fb.privacy_budget

100.0

In [13]:
X = fb.datasets[-1]["training_data"]
y = fb.datasets[-1]["training_targets"]

In [14]:
def relu(x,deriv=False):
    if deriv==True:
        return x>0
    return x*(x>0)

In [15]:
layer0_weights = 2*np.random.random((3,4)) - 1
layer1_weights = 2*np.random.random((4,1)) - 1

In [16]:
for j in range(2):
    # Forward propagation
    layer1_inputs = relu(X @ layer0_weights)  ; layer1_inputs.block
    layer2_inputs = relu(layer1_inputs @ layer1_weights) ; layer2_inputs.block 
    
    # Calculate errors
    layer2_inputs_delta = (y - layer2_inputs)* relu(layer2_inputs,deriv=True) ; layer2_inputs_delta.block
    layer1_inputs_delta = (layer2_inputs_delta@(layer1_weights.T)) * relu(layer1_inputs,deriv=True) ; layer1_inputs_delta.block
    
    # Update weights
    layer1_weights  = layer1_inputs.T @ layer2_inputs_delta + layer1_weights   ; layer1_weights.block
    layer0_weights =  X.T @ layer1_inputs_delta  + layer0_weights  ; layer0_weights.block

In [17]:
layer0_weights_dp = layer0_weights.publish(sigma=2e5)
layer1_weights_dp = layer1_weights.publish(sigma=2e5)

In [20]:
# Layer 0 weights
print(layer0_weights_dp)

[[[ 0.57238225  0.19638305 -3.40602193 -2.46518172]
  [ 0.3825276  -5.73164443 -1.84110786 -1.59676036]
  [-6.85407294 -1.02472956  1.5077444   2.71352215]]]


In [21]:
# Layer 1 weights
print(layer1_weights_dp)

[[[-5.73340856]
  [-3.51900474]
  [-4.66245209]
  [ 0.81856155]]]


<hr>

Let's see how our privacy budget changed as a result of training for a single epoch:

In [19]:
fb.privacy_budget

68.93513349875309

And voila! We've trained a neural network using PySyft's adversaril differential privacy system and its secure multiparty computation system working in tandem.


<hr>
